## GLAM fit. Value based. Gamma parameter limited to 0 - 1 . (Marius Usher recommendation)

In [1]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# Individual GLAM estimation and out of sample prediction

## Load data

In [4]:
# Load data
sufix = '_individual_Dislike_Inv_NoBin_Gamma_01_NUTS_31'
data = pd.read_csv('data/FF2018_data/GlamDataFF2018_Dislike_NoBin_Inv_31.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,2.05,1.00,0.745415,0.254585
1,0,1,1,3424,0.70,1.30,0.410720,0.589280
2,0,2,1,3691,1.30,1.75,0.330549,0.669451
3,0,3,0,8144,1.45,0.70,0.592345,0.407655
4,0,4,0,6559,1.00,1.00,0.641717,0.358283


## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/FF2018_data/GlamDataFF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1860 trials) and test (1860 trials) sets...


In [6]:
train_data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,1,3327,2.05,1.00,0.745415,0.254585
2,0,2,1,3691,1.30,1.75,0.330549,0.669451
4,0,4,0,6559,1.00,1.00,0.641717,0.358283
6,0,6,1,2470,1.45,1.90,0.329730,0.670270
8,0,8,1,4669,1.45,1.30,0.419431,0.580569
10,0,10,0,3301,1.90,1.90,0.605079,0.394921
12,0,12,1,3117,1.15,1.90,0.506602,0.493398
14,0,14,0,2754,1.75,1.45,0.596632,0.403368
16,0,16,1,3931,1.15,1.30,0.338392,0.661608
18,0,18,0,5674,1.30,1.60,0.410655,0.589345


## Individual GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_FF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(0, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_FF2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
Generating single subject models for 31 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 31 model(s) using NUTS...
  Fitting model 1 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [01:08<00:00, 175.96draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 31...


...

  Fitting model 30 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:18<00:00, 636.06draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 31 of 31...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [06:41<00:00, 29.91draws/s]
The acceptance probability does not match the target. It is 0.7051374736327186, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9990227801220327, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


/!\ Automatically setting parameter precision...


In [8]:
# Save parameter estimates
np.save(str('results/estimates/glam_FF2019_full'+sufix+'.npy'), glam_full.estimates)
pd.DataFrame(glam_full.estimates)

,SNR,b,gamma,p_error,s,t0,tau,v
0,138.97,1.0,0.59,0.05,0.006214,[0.0],4.36,0.000038
1,37.44,1.0,0.36,0.05,0.004059,[0.0],0.45,0.000048
2,159.08,1.0,0.98,0.05,0.007682,[0.0],0.86,0.000043
3,205.12,1.0,0.09,0.05,0.007723,[0.0],4.27,0.000035
4,191.66,1.0,0.22,0.05,0.010432,[0.0],4.83,0.000048
5,173.44,1.0,0.25,0.05,0.008257,[0.0],4.77,0.000046
6,136.86,1.0,0.20,0.05,0.008473,[0.0],1.40,0.000061
7,171.85,1.0,0.02,0.05,0.006690,[0.0],4.22,0.000041
8,144.42,1.0,0.01,0.05,0.008362,[0.0],1.01,0.000054
9,146.44,1.0,0.01,0.05,0.008484,[0.0],4.89,0.000051


In [9]:
full_params = pd.DataFrame(glam_full.estimates)
full_params.to_csv(str('results/params_estimates/params_glam_FF2019_full'+sufix+'.csv'))

In [10]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_FF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_FF2019_full'+ sufix +'.npy'))

# Compute WAICs
np.save(str('results/waic/glam_FF2019_full'+ sufix +'.npy'), glam_full.waic)

Computing WAIC scores for full model...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:168: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.py:219: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/pymc3/stats.

In [11]:
waic_values = pd.DataFrame(glam_full.waic)
waic_values.to_csv(str('results/params_estimates/waic_estimate'+sufix+'.csv'))

In [12]:
glam_full.waic

array([[1.06066384e+03, 0.00000000e+00, 1.67454378e+00, 1.00000000e+00],
       [1.85387738e+03, 0.00000000e+00, 3.86804975e+02, 1.00000000e+00],
       [1.08085812e+03, 0.00000000e+00, 2.57491502e+00, 1.00000000e+00],
       [1.08736582e+03, 0.00000000e+00, 1.73239806e+00, 1.00000000e+00],
       [1.05226327e+03, 0.00000000e+00, 1.60429593e+00, 1.00000000e+00],
       [1.06347708e+03, 0.00000000e+00, 1.87207163e+00, 1.00000000e+00],
       [1.06296887e+03, 0.00000000e+00, 2.37683162e+00, 1.00000000e+00],
       [1.02442691e+03, 0.00000000e+00, 2.07156083e+00, 1.00000000e+00],
       [1.08361385e+03, 0.00000000e+00, 2.82062177e+00, 1.00000000e+00],
       [1.04019672e+03, 0.00000000e+00, 2.43471250e+00, 1.00000000e+00],
       [1.04725320e+03, 0.00000000e+00, 3.05315877e+00, 1.00000000e+00],
       [1.03147323e+03, 0.00000000e+00, 1.61708076e+00, 1.00000000e+00],
       [3.57837893e+03, 0.00000000e+00, 1.24621002e+03, 1.00000000e+00],
       [1.09299544e+03, 0.00000000e+00, 2.04047627e

In [13]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_FF2019_full'+ sufix +'.npy'), glam_full.loo)

AttributeError: 'list' object has no attribute 'nchains'

In [14]:
glam_full.loo

AttributeError: 'GLAM' object has no attribute 'loo'

In [15]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_FF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_FF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1860 trials) with new data (1860 trials)...


,subject,trial,repeat,choice,rt,item_value_0,gaze_0,item_value_1,gaze_1
0,0.0,0.0,0.0,1.0,2806.0,0.7,0.41072,1.3,0.58928
1,0.0,0.0,1.0,1.0,2645.0,0.7,0.41072,1.3,0.58928
2,0.0,0.0,2.0,1.0,2592.0,0.7,0.41072,1.3,0.58928
3,0.0,0.0,3.0,1.0,3192.0,0.7,0.41072,1.3,0.58928
4,0.0,0.0,4.0,1.0,2660.0,0.7,0.41072,1.3,0.58928


## 2. Plot fit

In [ ]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
glam.plots_pretty.plot_fit(test_data, [glam_full.prediction]);

#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

## [END] 